In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model

In [2]:
BP_name = 'SBP'

In [4]:
df_PPG = pd.read_csv('../data/PPG_features.csv')
df_ECG = pd.read_csv('../data/ECG_features.csv')
df_PAT = pd.read_csv('../data/PAT_features.csv')

df = pd.concat([df_PPG, df_ECG], axis=1, join="inner")
df = pd.concat([df, df_PAT], axis=1, join="inner")

df = df.loc[:, ~df.columns.duplicated()].copy()

In [5]:
df_PPG = pd.read_csv('../data/PPG_features_aug.csv')
df_ECG = pd.read_csv('../data/ECG_features_aug.csv')
df_PAT = pd.read_csv('../data/PAT_features_aug.csv')

df_aug = pd.concat([df_PPG, df_ECG], axis=1, join="inner")
df_aug = pd.concat([df_aug, df_PAT], axis=1, join="inner")

df_aug = df_aug.loc[:, ~df_aug.columns.duplicated()].copy()

In [14]:
from MOLLIE_session import MOLLIE_session
from Dataset_utils import BP_est_dataset
from regression_model_funcs import *

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
from sklearn.preprocessing import StandardScaler
# import shap

In [22]:
model_name = 'LASSO'

MOLLIE = MOLLIE_session()
BP_est = BP_est_dataset(df = df,df_aug = df_aug, study = MOLLIE)
BP_est.calibrate_dataset()
BP_est.drop_collinear_features(VIF_max = 10)
BP_est.ID_iterator(restart=True)

### PERFORM LOSOCV

# for ii in range(len(BP_est.Original.df['ID'].unique())):
for ii in range(2):
    print(ii)
    BP_est.ID_iterator(val=ii+1)
    (X_train, X_test, y_train, y_test) = BP_est.return_LOSO_train_test_data()
    ### Standardise 
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test= sc.transform(X_test)
    ### LASSO + OLS
    cv = BP_est.return_inner_cv()
    mdl = linear_regression(X_train=X_train, y_train=y_train, method = 'LASSO', cv = cv)
    BP_est.results['LASSO'].update_results(mdl,X_test=X_test, y_test=y_test, X_train=X_train, y_train=y_train) 
    ### RF
#     cv = BP_est.return_inner_cv()
#     mdl = regression_funcs.RF_regression(X_train=X_train, y_train=y_train, run_hyper_param_tuning=False, cv = None)
#     BP_est.results['RF'].update_results(mdl,X_test=X_test, y_test=y_test, X_train=X_train, y_train=y_train) 
    
BP_est.pickle_class(file_name='a')

0
HERE
1
HERE


PicklingError: Can't pickle <class 'Dataset_utils.BP_est_dataset'>: it's not the same object as Dataset_utils.BP_est_dataset

In [17]:
mdl.cv = None

In [16]:
import copy

model_copy = copy.deepcopy(mdl)

TypeError: cannot pickle 'generator' object

In [ ]:
BP_est.pickle_class(file_name='a')

In [ ]:
BP_est.results['LASSO'].mdl_store[1].cv

In [ ]:
mdl.cv

In [ ]:
# Load data


BP_est = 